# 04 — Statistical Analysis of Demographic Disparities

Quantify associations between demographic factors and spatial accessibility.

**Methods:**
1. OLS regression (baseline)
2. Spatial autocorrelation tests (Moran's I)
3. Geographically Weighted Regression (GWR)
4. Local Indicators of Spatial Association (LISA)

In [ ]:
import sys
sys.path.insert(0, "..")

import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt

from src.config import DATA_OUTPUTS
from src.statistics import (
    run_ols_regression,
    run_gwr,
    compute_morans_i,
    lisa_analysis,
)

## 4.1 Load Accessibility Scores and Demographics

In [ ]:
analysis_gdf = gpd.read_file(DATA_OUTPUTS / "pa_accessibility_scores.gpkg")
analysis_gdf.shape, analysis_gdf.head()

## 4.2 Baseline OLS Regression

In [ ]:
ols_result = run_ols_regression(analysis_gdf)
print(ols_result["summary"])
ols_result["coefficients"].head()

## 4.3 Global Moran's I — Spatial Autocorrelation

In [ ]:
morans = compute_morans_i(analysis_gdf, variable="accessibility_score")
morans

## 4.4 Geographically Weighted Regression (GWR)

In [ ]:
try:
    gwr_result = run_gwr(analysis_gdf)
    gwr_result["local_coefficients"].head()
except Exception as exc:
    gwr_result = None
    print(f"GWR not available in this environment: {exc}")

## 4.5 LISA Analysis — Local Clustering

In [ ]:
lisa_gdf = lisa_analysis(analysis_gdf, variable="accessibility_score")
lisa_gdf[["geoid", "lisa_cluster", "lisa_p_value"]].head()

## 4.6 Map Local GWR Coefficients

In [ ]:
if gwr_result is not None:
    coef_col = [c for c in gwr_result["local_coefficients"].columns if c != "const"][0]
    lisa_gdf[f"gwr_{coef_col}"] = gwr_result["local_coefficients"][coef_col].reindex(lisa_gdf.index)
    ax = lisa_gdf.plot(column=f"gwr_{coef_col}", figsize=(9, 6), legend=True)
    ax.set_title(f"Local GWR coefficient: {coef_col}")
    ax.axis("off")
else:
    print("Skipping local GWR coefficient map because GWR did not run.")

## 4.7 Model Comparison (OLS vs GWR)

In [ ]:
comparison = {
    "ols_r_squared": ols_result["r_squared"],
    "ols_aic": ols_result["aic"],
    "gwr_aic": gwr_result["aic"] if gwr_result is not None else None,
}
comparison